# Add sizes to the catalog of low-z galaxies

Get diameters from NED and add them to the CSV file.

https://astroquery.readthedocs.io/en/latest/ipac/ned/ned.html#fetching-other-data-tables-for-an-object

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astroquery.ipac.ned import Ned

In [ ]:
speed_of_light = 3e18 # A/s

## Read the list of low-z galaxies

In [ ]:
fnm = "lowz_galaxies.csv"
gal_name = np.loadtxt(fnm, dtype='str', usecols=(0), delimiter=',')
gal_ra = np.loadtxt(fnm, dtype='float', usecols=(1), delimiter=',')
gal_dec = np.loadtxt(fnm, dtype='float', usecols=(2), delimiter=',')
gal_z = np.loadtxt(fnm, dtype='float', usecols=(3), delimiter=',')

## Get diameter data from NED

For example, this is how to retrieve it.

In [ ]:
# result_table = Ned.get_table("NGC 224", table='diameters')
result_table = Ned.get_table("IC 5371", table='diameters')
# result_table = Ned.get_table("IC 5377", table='diameters')
# result_table = Ned.get_table("NGC 7807", table='diameters')
result_table

In [ ]:
# result_table.colnames

In [ ]:
# temp = np.asarray(result_table['Frequency targeted'], dtype='str')
# for tmp in temp:
#     if tmp.find('isophotal') > -1:
#         print(tmp)

### Decide which diameter values to use

Want to be consistent.

To narrow the measurements down to a single diameter value: 

1. optical-NIR bands
2. the most recent reference
3. the smallest estimate (be conservative)

Record in the table:

1. NED Major Axis (its always diameter in arcsec not radius, not semi-)
2. NED Axis Ratio
3. NED Position Angle (always degrees east of north
4. NED Frequency
5. Refcode

### Demonstrate diameter selection for one galaxy

Choose the diameter using band, reference, and size for just one galaxy.

Convert the NED Frequency to a wavelength.

In [ ]:
frequencies = np.asarray(result_table['NED Frequency'], dtype='float64')
wavelengths = speed_of_light / frequencies

Get the year out of the reference code.

In [ ]:
refcodes = np.asarray(result_table['Refcode'], dtype='str')
temp = []
for refcode in refcodes:
    temp.append(refcode[0:4])
years = np.asarray(temp, dtype='int')
del temp

Get the major axes measurements.

In [ ]:
ned_major_axes = np.asarray(result_table['NED Major Axis'], dtype='float64')

Write out the bands, years, and axes that will be used to make the selection.

In [ ]:
for i in range(len(result_table)):
    print(wavelengths[i], years[i], ned_major_axes[i])

Impose the constraint of optical band and most recent reference.

Then choose the smallest major axis estimate.

Print the full record for the winning row.

In [ ]:
tx = np.where((wavelengths > 3000) & (wavelengths < 30000) & (years == np.max(years)))[0]
mx = np.argmin(ned_major_axes[tx])
result_table[tx[mx]]

The values added to the low-z galaxy table for this object would be the following.

In [ ]:
print('NED Major Axis: ', result_table['NED Major Axis'][tx[mx]])
print('NED Axis Ratio: ', result_table['NED Axis Ratio'][tx[mx]])
print('NED Position Angle: ', result_table['NED Position Angle'][tx[mx]])
print('NED Frequency: ', result_table['NED Frequency'][tx[mx]])
print('Refcode: ', result_table['Refcode'][tx[mx]])

In [ ]:
del result_table, tx, mx
del frequencies, wavelengths, refcodes, years, ned_major_axes

### Get diameters for all listed galaxies

Results will be written out to file "lowz_galaxies_with_diameters.csv".

Doing the NED query for thousands of objects will take a while.

Write to the file as data is retrieved.
Set up the write so that the process can be restarted from where it left off in case there is a NED connection issue.

If the new file already exists, read in the list of galaxy names to `check_done_gals` and make a copy to protect it.
Then open the file for writing in append mode.

If the new file does not exist, create `check_done_gals` as a dummy variable to cross-match to.
Then open the file for writing and write the header first.

In [ ]:
fnm = 'lowz_galaxies_with_diameters.csv'
if os.path.exists(fnm):
    print('Does exist: ', fnm)
    check_done_gals = np.loadtxt(fnm, dtype='str', usecols=(0), delimiter=',')
    os.system('cp ' + fnm + ' PROTECT_' + fnm)
    fout = open(fnm, 'a')
else:
    print('Does not exist: ', fnm)
    check_done_gals = np.asarray(['dummy1','dummy2','dummy3'], dtype='str')
    fout = open(fnm, 'w')
    fout.write('# name, ra, dec, redshift, '+\
               'major axis, axis ratio, position angle, '+\
               'frequency, refcode \n')
    
verbose = True

for i in range(len(gal_name)):
# for i in range(8000):
    tx = np.where(gal_name[i] == check_done_gals)[0]

    if len(tx) > 0:
        if verbose:
            print('already done: ', i, gal_name[i])    
    elif len(tx) == 0:
        try:
            if verbose:
                print('trying: ', i, gal_name[i])
            result_table = Ned.get_table(gal_name[i], table='diameters')
            
            frequencies = np.asarray(result_table['NED Frequency'], dtype='float64')
            wavelengths = speed_of_light / frequencies
            
            refcodes = np.asarray(result_table['Refcode'], dtype='str')
            temp = []
            for refcode in refcodes:
                temp.append(refcode[0:4])
            years = np.asarray(temp, dtype='int')
            del temp
    
            ned_major_axes = np.asarray(result_table['NED Major Axis'], dtype='float64')
    
            tx = np.where((wavelengths > 3000) & (wavelengths < 30000) & (years == np.max(years)))[0]
            if len(tx) > 1:
                mx = np.argmin(ned_major_axes[tx])
                ned_major_axis = float(result_table['NED Major Axis'][tx[mx]])
                ned_axis_ratio = float(result_table['NED Axis Ratio'][tx[mx]])
                ned_pos_angle = float(result_table['NED Position Angle'][tx[mx]])
                ned_frequency = float(result_table['NED Frequency'][tx[mx]])
                ned_refcode = result_table['Refcode'][tx[mx]]
                del mx
            elif len(tx) == 1:
                ned_major_axis = float(result_table['NED Major Axis'][tx[0]])
                ned_axis_ratio = float(result_table['NED Axis Ratio'][tx[0]])
                ned_pos_angle = float(result_table['NED Position Angle'][tx[0]])
                ned_frequency = float(result_table['NED Frequency'][tx[0]])
                ned_refcode = result_table['Refcode'][tx[0]]
            elif len(tx) == 0:
                ned_major_axis = float(-99.99)
                ned_axis_ratio = float(-99.99)
                ned_pos_angle = float(-99.99)
                ned_frequency = float(-99.99)
                ned_refcode = '--'

            del result_table, frequencies, wavelengths, refcodes, years, ned_major_axes, tx
            
        except:
            if verbose:
                print('except failure: ', gal_name[i])
            ned_major_axis = float(-99.99)
            ned_axis_ratio = float(-99.99)
            ned_pos_angle = float(-99.99)
            ned_frequency = float(-99.99)
            ned_refcode = '--'

        strout = gal_name[i] + ', '
        strout += str(gal_ra[i]) + ', ' + str(gal_dec[i]) + ', '
        strout += str(gal_z[i]) + ', '
        strout += str(ned_major_axis) + ', '
        strout += str(ned_axis_ratio) + ', '
        strout += str(ned_pos_angle) + ', '
        strout += str(ned_frequency) + ', '
        strout += ned_refcode + ' \n'
        fout.write(strout)
        del ned_major_axis, ned_axis_ratio, ned_pos_angle, ned_frequency
        del ned_refcode, strout

fout.close()

In [ ]:
del fnm, verbose
del gal_name, gal_ra, gal_dec, gal_z

## Wait, how often was a lack of band the issue?

In [ ]:
fnm = "lowz_galaxies_with_diameters.csv"
gal_name = np.loadtxt(fnm, dtype='str', usecols=(0), delimiter=',')
gal_ra = np.loadtxt(fnm, dtype='float', usecols=(1), delimiter=',')
gal_dec = np.loadtxt(fnm, dtype='float', usecols=(2), delimiter=',')
gal_z = np.loadtxt(fnm, dtype='float', usecols=(3), delimiter=',')
gal_ma = np.loadtxt(fnm, dtype='float', usecols=(4), delimiter=',')
gal_ar = np.loadtxt(fnm, dtype='float', usecols=(5), delimiter=',')
gal_pa = np.loadtxt(fnm, dtype='float', usecols=(6), delimiter=',')
gal_fr = np.loadtxt(fnm, dtype='float', usecols=(7), delimiter=',')
gal_rc = np.loadtxt(fnm, dtype='str', usecols=(8), delimiter=',')

In [ ]:
print(len(gal_name))

In [ ]:
tx = np.where(gal_ma < 0.0)[0]
print(len(tx))

In [ ]:
gal_fail_flag = np.zeros(len(gal_name), dtype='int')
# -1 : no NED info retrieved
#  0 : major axis was found already
#  1 : there was a major axis in optical band
#  2 : there was a major axis in a non-optical band
#  3 : there was a major axis estimate
#  4 : there was no major axis estimate

temp_band = []

for i in range(len(gal_name)):
    if gal_ma[i] < 0.0:
        gal_fail_flag[i] = 5
        try:
            # print(i, gal_name[i])
            result_table = Ned.get_table(gal_name[i], table='diameters')

            frequencies = np.asarray(result_table['NED Frequency'], dtype='float64')
            wavelengths = speed_of_light / frequencies

            refcodes = np.asarray(result_table['Refcode'], dtype='str')
            temp = []
            for refcode in refcodes:
                temp.append(refcode[0:4])
            years = np.asarray(temp, dtype='int')
            del temp

            ned_major_axes = np.asarray(result_table['NED Major Axis'], dtype='float64')

            tx1 = np.where(((wavelengths > 3000) | (wavelengths < 30000)) & \
                           (ned_major_axes > 0.0))[0]
            tx2 = np.where(((wavelengths <= 3000) | (wavelengths >= 30000)) & \
                           (ned_major_axes > 0.0))[0]
            tx3 = np.where(ned_major_axes > 0.0)[0]
            if len(tx3) > 0:
                gal_fail_flag[i] = 3
            if len(tx2) > 0:
                gal_fail_flag[i] = 2
                
            if len(tx1) > 0:
                gal_fail_flag[i] = 1
                for x in tx1:
                    temp_band.append(result_table['NED Frequency'][x])
            else:
                gal_fail_flag[i] = 4

        except:
            gal_fail_flag[i] = -1

In [ ]:
values, counts = np.unique(gal_fail_flag, return_counts = True)
for value, count in zip(values, counts):
    print(value, count)

In [ ]:
bands = np.asarray(temp_band, dtype='str')
values, counts = np.unique(bands, return_counts = True)
for value, count in zip(values, counts):
    print('%22s %10.2f %5i' % (value, speed_of_light / float(value), count))

In [ ]:
del temp_band, bans, values, counts, gal_fail_flag

### Fill in where we missed

In 440 low-z galaxies, there was an optical/NIR size available. Go back through and write it out.

In [ ]:
fnm = 'lowz_galaxies_with_diameters_fix.csv'
fout = open(fnm, 'w')
fout.write('# name, ra, dec, redshift, '+\
           'major axis, axis ratio, position angle, '+\
           'frequency, refcode \n')

tally1 = 0
tally2 = 0
tally3 = 0
tally4 = 0

for i in range(len(gal_name)):
    if gal_ma[i] < 0.0:
        try:
            result_table = Ned.get_table(gal_name[i], table='diameters')
            
            frequencies = np.asarray(result_table['NED Frequency'], dtype='float64')
            wavelengths = speed_of_light / frequencies
            ned_major_axes = np.asarray(result_table['NED Major Axis'], dtype='float64')

            tx1 = np.where(((wavelengths > 3000) | (wavelengths < 30000)) & \
                           (ned_major_axes > 0.0))[0]                
            if len(tx1) > 1:
                mx = np.argmin(ned_major_axes[tx1])
                ned_major_axis = float(result_table['NED Major Axis'][tx1[mx]])
                ned_axis_ratio = float(result_table['NED Axis Ratio'][tx1[mx]])
                ned_pos_angle = float(result_table['NED Position Angle'][tx1[mx]])
                ned_frequency = float(result_table['NED Frequency'][tx1[mx]])
                ned_refcode = result_table['Refcode'][tx1[mx]]
                del mx
                tally1 += 1 
            elif len(tx1) == 1:
                ned_major_axis = float(result_table['NED Major Axis'][tx1[0]])
                ned_axis_ratio = float(result_table['NED Axis Ratio'][tx1[0]])
                ned_pos_angle = float(result_table['NED Position Angle'][tx1[0]])
                ned_frequency = float(result_table['NED Frequency'][tx1[0]])
                ned_refcode = result_table['Refcode'][tx1[mx]]
                tally1 += 1
            else:
                ned_major_axis = float(-99.99)
                ned_axis_ratio = float(-99.99)
                ned_pos_angle = float(-99.99)
                ned_frequency = float(-99.99)
                ned_refcode = '--'
                tally2 += 1

            del result_table, frequencies, wavelengths, ned_major_axes, tx1
            
        except:
            ned_major_axis = float(-99.99)
            ned_axis_ratio = float(-99.99)
            ned_pos_angle = float(-99.99)
            ned_frequency = float(-99.99)
            ned_refcode = '--'
            tally3 += 1

    else:
        ned_major_axis = gal_ma[i]
        ned_axis_ratio = gal_ar[i]
        ned_pos_angle = gal_pa[i]
        ned_frequency = gal_fr[i]
        ned_refcode = gal_rc[i]
        tally4 += 1

    strout = gal_name[i] + ', '
    strout += str(gal_ra[i]) + ', ' + str(gal_dec[i]) + ', '
    strout += str(gal_z[i]) + ', '
    strout += str(ned_major_axis) + ', '
    strout += str(ned_axis_ratio) + ', '
    strout += str(ned_pos_angle) + ', '
    strout += str(ned_frequency) + ', '
    strout += ned_refcode + ' \n'
    fout.write(strout)
    del ned_major_axis, ned_axis_ratio, ned_pos_angle, ned_frequency
    del ned_refcode, strout

fout.close()

In [ ]:
print(tally1, tally2, tally3, tally4)

In [ ]:
print(tally1 + tally2 + tally3 + tally4)

In [ ]:
del tally1, tally2, tally3, tally4
del gal_name, gal_ra, gal_dec, gal_z
del gal_ma, gal_ar, gal_pz, gal_fr, gal_rc

## Explore duplicates

In [ ]:
fnm = "lowz_galaxies_with_diameters_fix.csv"
gal_name = np.loadtxt(fnm, dtype='str', usecols=(0), delimiter=',')
gal_ra = np.loadtxt(fnm, dtype='float', usecols=(1), delimiter=',')
gal_dec = np.loadtxt(fnm, dtype='float', usecols=(2), delimiter=',')
gal_z = np.loadtxt(fnm, dtype='float', usecols=(3), delimiter=',')
gal_ma = np.loadtxt(fnm, dtype='float', usecols=(4), delimiter=',')
gal_ar = np.loadtxt(fnm, dtype='float', usecols=(5), delimiter=',')
gal_pa = np.loadtxt(fnm, dtype='float', usecols=(6), delimiter=',')
gal_fr = np.loadtxt(fnm, dtype='float', usecols=(7), delimiter=',')
gal_rc = np.loadtxt(fnm, dtype='str', usecols=(8), delimiter=',')

In [ ]:
radius = 30.0 #arcsec
gal_nn = np.zeros(len(gal_name), dtype='int')
gal_nn_ys = np.zeros(len(gal_name), dtype='int')
gal_nn_ns = np.zeros(len(gal_name), dtype='int')

for i, name in enumerate(gal_name):
    temp = np.sqrt(((np.cos(np.deg2rad(gal_dec[i]))) * (gal_ra[i] - gal_ra))**2 + 
                   (gal_dec[i] - gal_dec)**2)
    tx = np.where((temp > 0) & (temp < radius/3600.0))[0]
    tx1 = np.where((temp > 0) & (temp < radius/3600.0) & (gal_ma > 0.0))[0]
    tx2 = np.where((temp > 0) & (temp < radius/3600.0) & (gal_ma <= 0.0))[0]
    gal_nn[i] = len(tx)
    gal_nn_ys[i] = len(tx1)
    gal_nn_ns[i] = len(tx2)
    del temp, tx, tx1, tx2

In [ ]:
print('Number of neighbors.')
values, counts = np.unique(gal_nn, return_counts = True)
for value, count in zip(values, counts):
    print(value, count)

print('Number of neighbors that have a size.')
values, counts = np.unique(gal_nn_ys, return_counts = True)
for value, count in zip(values, counts):
    print(value, count)

print('Number of neighbors that do not have a size.')
values, counts = np.unique(gal_nn_ns, return_counts = True)
for value, count in zip(values, counts):
    print(value, count)

In [ ]:
tx1 = np.where(gal_ma <= 0.0)[0]
print('Number with no size: ', len(tx1))
tx2 = np.where((gal_ma <= 0.0) & (gal_nn_ys > 0))[0]
print('... but do have neighbor with size:', len(tx2))

In [ ]:
tx = np.where((gal_ma <= 0.0) & (gal_nn_ys > 0))[0]
sx = np.argsort(gal_name[tx])
for x in sx:
    i = tx[x]
    print(gal_name[i], gal_ra[i], gal_dec[i], gal_ma[i])
    temp = np.sqrt(((np.cos(np.deg2rad(gal_dec[i]))) * (gal_ra[i] - gal_ra))**2 + 
                   (gal_dec[i] - gal_dec)**2)
    tx1 = np.where((temp > 0) & (temp < radius/3600.0) & (gal_ma > 0.0))[0]
    for j in tx1:
        tmp = '-'
        if temp[j]*3600.0 < gal_ma[j]:
            tmp = 'IN'
        print(' # ', gal_name[j], gal_ra[j], gal_dec[j], gal_ma[j], ' off=', np.round(temp[j]*3600.0,1), tmp)
    print(' ')

In [ ]:
# ix = np.where(gal_nn == 5)[0]
# for i in ix:
#     print(gal_name[i], gal_ma[i])
#     temp = np.sqrt(((np.cos(np.deg2rad(gal_dec[i]))) * (gal_ra[i] - gal_ra))**2 + 
#                    (gal_dec[i] - gal_dec)**2)
#     tx = np.where((temp > 0) & (temp < radius/3600.0))[0]
#     for x in tx:
#         print('   ', gal_name[x], gal_ma[x])